# Aula 8 - Visconde

Leandro Carísio Fernandes

## Setup

In [1]:
TOTAL_PERGUNTAS = 150

DIR_AULA8 = "/content/drive/My Drive/IA024A-Redes_Neurais_NLP/Aula8-Visconde/"

# Se True, gera o índice invertido e salva em um pickle. Se False, apenas recupera o pickle do drive
GERAR_INDICE_INVERTIDO = False
ARQUIVO_INDICE_INVERTIDO = 'iidx_artigos_segmentados' # A função salva em 2 arquivos, então ela mesma coloca a extensão pickle

# Se True, gera as respostas. Se False, apenas recupera o pickle do drive
AGREGAR_DOCUMENTOS = False
ARQUIVO_AGREGADO = 'respostas_obtidas.pickle'

In [2]:
from getpass import getpass
GROQ_API = getpass("API groq")

API groq··········


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install Groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


# Download da base de dados

Link para a base IIRC: https://github.com/jferguson144/IIRC-baseline

Baixa arquivo iirc.tar.gz e context_articles.tar.gz conforme setup.sh do IIRC: https://github.com/jferguson144/IIRC-baseline/blob/main/setup.sh

In [5]:
!wget http://jamesf-incomplete-qa.s3.amazonaws.com/iirc.tar.gz
!tar -xzf iirc.tar.gz
!rm iirc.tar.gz
!cd iirc

!wget http://jamesf-incomplete-qa.s3.amazonaws.com/context_articles.tar.gz
!tar -xzf context_articles.tar.gz
!rm context_articles.tar.gz

!wget https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json

--2024-05-06 16:49:22--  http://jamesf-incomplete-qa.s3.amazonaws.com/iirc.tar.gz
Resolving jamesf-incomplete-qa.s3.amazonaws.com (jamesf-incomplete-qa.s3.amazonaws.com)... 52.92.185.65, 52.92.233.169, 52.92.179.129, ...
Connecting to jamesf-incomplete-qa.s3.amazonaws.com (jamesf-incomplete-qa.s3.amazonaws.com)|52.92.185.65|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5713947 (5.4M) [application/x-gzip]
Saving to: ‘iirc.tar.gz’

iirc.tar.gz         100%[===================>]   5.45M  9.70MB/s    in 0.6s    

2024-05-06 16:49:23 (9.70 MB/s) - ‘iirc.tar.gz’ saved [5713947/5713947]

--2024-05-06 16:49:24--  http://jamesf-incomplete-qa.s3.amazonaws.com/context_articles.tar.gz
Resolving jamesf-incomplete-qa.s3.amazonaws.com (jamesf-incomplete-qa.s3.amazonaws.com)... 52.92.185.65, 52.92.233.169, 52.92.179.129, ...
Connecting to jamesf-incomplete-qa.s3.amazonaws.com (jamesf-incomplete-qa.s3.amazonaws.com)|52.92.185.65|:80... connected.
HTTP request sent, awaiting r

In [6]:
import json

dev_set = json.load(open('iirc/dev.json','r'))
test_set = json.load(open('iirc_test.json', 'r'))
context_articles = json.load(open("context_articles.json",'r'))

## Entendendo o dataset

context_articles são os artigos que deverão ser consultados. É um dict onde a chave é o título do artigo. Por exemplo:

In [7]:
print(f'Total de artigos: ${len(context_articles)}')
print('5 títulos quaisquer:')
print(list(context_articles.keys())[0:5])

Total de artigos: $56550
5 títulos quaisquer:
['san diego padres', 'john bruce (historiographer)', 'jack nicholson', 'british raj', 'dabu county']


O conjunto de teste tem um monte de perguntas e respostas. Ele indica a pergunta, a resposta esperada e de onde aquela resposta veio. Por exemplo:

In [8]:
i = 0 # i = 3 # Dá pra testar isso com qualquer tipo de resposta span. Para outros tipos, tem que adaptar
print('Primeira pergunta do conjunto de teste:')
print('Pergunta:', test_set[i]['questions'][0]['question'])
print('Resposta:', test_set[i]['questions'][0]['answer']['answer_spans'][0]['text']) # Isso só funciona pois já sabemos que é do tipo span
print('De onde veio a resposta:', test_set[i]['questions'][0]['answer']['answer_spans'][0]['passage'])
print('---------------------')
print('Artico no context_articles:')
context_articles[test_set[i]['questions'][0]['answer']['answer_spans'][0]['passage']]

Primeira pergunta do conjunto de teste:
Pergunta: What is Zeus know for in Greek mythology?
Resposta: sky and thunder god
De onde veio a resposta: zeus
---------------------
Artico no context_articles:


'Zeus (British English , North American English ; , <i>Zeús</i> ) is the <a href="sky%20father">sky</a> and <a href="thunder%20god">thunder god</a> in <a href="ancient%20Greek%20religion">ancient Greek religion</a>, who rules as <a href="king%20of%20the%20gods">king of the gods</a> of <a href="Mount%20Olympus">Mount Olympus</a>. His name is <a href="cognate">cognate</a> with the first element of his <a href="ancient%20Roman%20religion">Roman</a> <a href="interpretatio%20graeca">equivalent</a> <a href="Jupiter%20%28mythology%29">Jupiter</a>. His mythologies and powers are similar, though not identical, to those of Indo-European deities such as <a href="Jupiter%20%28mythology%29">Jupiter</a>, <a href="Perk%C5%ABnas">Perkūnas</a>, <a href="Perun">Perun</a>, <a href="Indra">Indra</a> and <a href="Thor">Thor</a>.\n\nZeus is the child of <a href="Cronus">Cronus</a> and <a href="Rhea%20%28mythology%29">Rhea</a>, the youngest of his siblings to be born, though sometimes reckoned the eldest as 

## Organizando os dados

Organizando o conjunto de teste para gerar um array de pergunta e resposta esperada.

Imprime as 10 primeiras perguntas e respostas para checar.

Daqui dá pra pegar as 150 questões e respostas pra gente testar no final.

In [9]:
perg_resposta_esperada = []

for i in range(len(test_set)):
  pr = test_set[i]['questions'][0]

  pergunta = pr['question']
  resposta_obj = pr['answer']
  tipo_resposta = resposta_obj['type']

  if tipo_resposta == 'binary' or tipo_resposta == 'value':
    resposta = resposta_obj['answer_value']
  elif tipo_resposta == 'span':
    resposta = resposta_obj['answer_spans'][0]['text']
  elif tipo_resposta == 'none':
    resposta = 'none'
  else:
    resposta = 'Não pode chegar aqui, os tipos parece que são só binary/value/span/none!'
    print(tipo_resposta)

  perg_resposta_esperada.append({"pergunta": pergunta, "resposta": resposta})

  if i < 10:
    print(f"{i}: {pergunta}\n{resposta}")
    print('----------------------------------------------------------------------')



0: What is Zeus know for in Greek mythology?
sky and thunder god
----------------------------------------------------------------------
1: How long had the First World War been over when Messe was named aide-de-camp?
5
----------------------------------------------------------------------
2: How long had Angela Scoular been acting professionally when she appeared in the movie "On Her Majesty's Secret Service"?
2
----------------------------------------------------------------------
3: What is the capacity of the stadium where Brunt returned to action after a torn ACL?
26,688
----------------------------------------------------------------------
4: In which country was Wilhelm Müller born?
Germany 
----------------------------------------------------------------------
5: In which Italian region did Pesce studied medicine?
Liguria
----------------------------------------------------------------------
6: What albums were ranked higher than "It Takes a Nation of Millions to Hold Us Back" i

# BM25

## Tratando os dados que serão indexados

In [10]:
import re

# Remover o html igual está na base IIRC é bem mais rápido que usar o BeuatifulSoup
# https://github.com/jferguson144/IIRC-baseline/blob/main/util.py
def remove_html(html):
  return re.sub("<[^>]*>", "", html).strip()

Sobre a base IIRC:

"Contains more than 13K questions over paragraphs from English Wikipedia that provide only partial information to answer them, with the missing information occurring in one or more linked documents. The questions were written by crowd workers who did not have access to any of the linked documents, leading to questions that have little lexical overlap with the contexts where the answers appear." [https://paperswithcode.com/dataset/iirc]

Assim, para a base de testes, vamos indexar todos os documentos relacionados à pergunta (ou seja, o documento e os linkados).

In [11]:
%%time
documents = []
all_titles = []

for item in test_set[:TOTAL_PERGUNTAS]:
    if item['title'].lower() not in all_titles:
        documents.append({
                "title": item['title'],
                "content": item["text"]
            }
        )
        all_titles.append(item['title'].lower())
    for link in item["links"]:
        if link['target'].lower() in context_articles and link['target'].lower() not in all_titles:
            documents.append({
                "title": link['target'],
                "content": context_articles[link['target'].lower()]
            })
            all_titles.append(link['target'].lower())
        # else:
            # print(link['target'].lower())

CPU times: user 70.8 ms, sys: 813 µs, total: 71.7 ms
Wall time: 153 ms


Limpa o HTML dos documentos:

In [12]:
artigos_sem_html = []

# Se for usar todos os context_articles para indexar:
#for titulo in context_articles.keys():
#  artigo_html = context_articles[titulo]
#  artigos_sem_html.append({"title": titulo, "content": remove_html(artigo_html)})

# Se for usar só os documentos que estão na base de teste
for doc in documents:
  titulo = doc['title']
  conteudo = doc['content']
  artigos_sem_html.append({"title": titulo, "content": remove_html(conteudo)})

Quebra cada artigo em grupos de 3 sentenças. Usa a biblioteca Spacy para separar as sentenças.

Código de origem: https://github.com/neuralmind-ai/visconde/blob/main/iirc_create_indices.ipynb

In [13]:
%%time
from tqdm import tqdm
import spacy
import pickle

nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

stride = 2
max_length = 3

def window(documents, stride=2, max_length=3):
  treated_documents = []

  for j, document in enumerate(tqdm(documents)):
    doc_text = document['content']
    doc = nlp(doc_text)
    sentences = [sent.text.strip() for sent in doc.sents]
    for i in range(0, len(sentences), stride):
      segment = ' '.join(sentences[i:i + max_length])
      treated_documents.append({
          "title": document['title'],
          "contents": document['title']+". "+segment,
          "segment": segment
      })
      if i + max_length >= len(sentences):
        break
  return treated_documents


artigos_segmentados = window(artigos_sem_html)

100%|██████████| 2164/2164 [00:59<00:00, 36.07it/s]

CPU times: user 1min 2s, sys: 1.18 s, total: 1min 3s
Wall time: 1min 9s


In [14]:
len(artigos_segmentados)

161738

## Implementação do BM25

In [15]:
%%time
from collections import Counter
import array
import pickle
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

# Definição de uma classe para índice invertido
class IndiceInvertido:

  # Recebe 'tokenizar', uma função tokenizadora
  def __init__(self):
    # Cria um índice invertido vazio
    self.indice = {}
    # Cria um índice de tamanho de documentos vazio
    self.tamanho_doc = {}
    # Guarda o total de documentos adicionados
    self.n_docs = 0
    # Tokenizador
    self.stemmer = PorterStemmer()

  def tokenizar(self, texto):
    # Tokeniza
    tokens = word_tokenize(texto)
    # Converte para minúsculo, filtra stopwords e passa pelo Porter Stemer
    tokens = [self.stemmer.stem(token.lower()) for token in tokens if token.lower() not in stopwords.words('english')]
    # Converte para minúsculo, não filtra stopwords e passa pelo Porter Stemer
    #tokens = [self.stemmer.stem(token.lower()) for token in tokens]

    return tokens

  def adiciona_doc(self, id_doc, conteudo_doc=None):
    tokens = self.tokenizar(conteudo_doc)

    contador_tokens_do_documento = Counter(tokens)
    for token, n_ocorrencias in contador_tokens_do_documento.items():
      self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})['id_doc'].append(id_doc)
      self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})['n_ocorrencias'].append(n_ocorrencias)

    self.n_docs += 1
    self.tamanho_doc[id_doc] = len(tokens)

class BM25:

  def __init__(self, indiceInvertido=IndiceInvertido(), k1 = 0.9, b = 0.4, bias_adicionar_ao_idf = 0):
    self.indiceInvertido = indiceInvertido
    self.bias_adicionar_ao_idf = bias_adicionar_ao_idf
    self.calcula_tam_medio_doc_no_indice()
    self.k1 = k1
    self.b = b
    self.precalcula_idf()
    self.reinicia_score_dos_indices()

  def reinicia_score_dos_indices(self):
    for token in self.indiceInvertido.indice.keys():
      self.indiceInvertido.indice[token].pop('score', None)

  def calcula_tam_medio_doc_no_indice(self):
    self.avgdl = sum(self.indiceInvertido.tamanho_doc.values()) / self.indiceInvertido.n_docs

  def precalcula_idf(self):
    # Número de documento do corpus está presente no objeto indiceInvertido
    N = self.indiceInvertido.n_docs
    # Varre todos os tokens do índice. Os tokens são as chaves do indiceInvertido.indice
    for token in self.indiceInvertido.indice.keys():
      # O número de documentos que possui o token é calculado pelo tamanho da lista de id_doc:
      n_doc_token = len(self.indiceInvertido.indice[token]['id_doc'])
      # Isso já é o suficiente pra calcular o idf
      idf_token = math.log( ((self.indiceInvertido.n_docs - n_doc_token + 0.5)/(n_doc_token + 0.5)) + self.bias_adicionar_ao_idf )
      # E agora, vamos colocar essa informação no índice
      self.indiceInvertido.indice[token]['idf'] = idf_token

  def calcula_score_para_um_token_e_salva(self, token):
    # O cálculo do BM25 para determinada query é a multiplicação do idf pela frequência do termo no documento * (k1 + 1)
    # Além disso, é dividido pela frequencia do termo no documento + k1 * (1 - b + b * tamanho_doc/avgdl)
    idf = self.indiceInvertido.indice[token]['idf']
    # Juntando tudo, podemos calcular o score pelo BM25
    zip_id_freq = zip(self.indiceInvertido.indice[token]['id_doc'], self.indiceInvertido.indice[token]['n_ocorrencias'])
    bm25 = array.array("f", [ idf * freq_token_no_doc * (self.k1 + 1) / (freq_token_no_doc + self.k1 * (1 - self.b + self.b * self.indiceInvertido.tamanho_doc[id_doc] / self.avgdl)) for (id_doc, freq_token_no_doc) in zip_id_freq ])
    # Salva o bm25 no índice
    self.indiceInvertido.indice[token]['score'] = bm25

  def tokenizar(self, query):
    return self.indiceInvertido.tokenizar(query)

  def pesquisar(self, query):
    # Tokeniza a query
    tokens = self.tokenizar(query)

    # Se não tem token para ser pesquisado, retorna conjunto vazio
    if (len(tokens) == 0):
      return []

    # Guarda um dicionário onde a chave é o id do documento e o valor é o score desse documento para a query pesquisada
    docs_retornado_com_score = Counter({})

    # Faz a pesquisa de documentos. Para isso iteramos todos os tokens da query
    for token in tokens:
      # É possível que a query contenha algum termo que não foi indexado. Se isso ocorrer,
      # entende-se que a frequência desse token em qualquer documento é 0, já que não pode ser encontrado
      if token not in self.indiceInvertido.indice:
        continue

      # Pega a lista de documentos que será analisado
      docs_que_tem_token = self.indiceInvertido.indice[token]['id_doc']

      # Se for a primeira vez que esse token é pesquisado, é necessário calcular o score relacionado
      # a ele e salvar. Se já tiver sido feito antes, já podemos buscar o cálculo pronto (que funciona
      # como um cache. Isso é útil no caso de várias pesquisas seguidas)
      if 'score' not in self.indiceInvertido.indice[token].keys():
        self.calcula_score_para_um_token_e_salva(token)
      score_dos_docs_deste_token = self.indiceInvertido.indice[token]['score']

      # Agora já temos calculado o score de todos os documentos desse token. Só adiciona ao acumulador de score atual
      # docs_retornado_com_score += score_dos_docs_deste_token -> Se fosse usar dict direto no índice seria assim, mas a memória não está aguentando guardar os scores de ambos
      for id_doc, score_par_doc_token in zip(docs_que_tem_token, score_dos_docs_deste_token):
        docs_retornado_com_score[id_doc] += score_par_doc_token

    # Agora converte esse dict em uma lista de tuplas com a chave (id_doc) e valor (score_do_doc)
    docs_com_score = list(docs_retornado_com_score.items())

    # E ordena do mais relevante para o menos relevante
    return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


CPU times: user 1.27 s, sys: 446 ms, total: 1.72 s
Wall time: 2.75 s


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Testa uma tokenização qualquer:

In [16]:
iidx = IndiceInvertido()
iidx.tokenizar('What is Zeus know for in Greek mythology?')

['zeu', 'know', 'greek', 'mytholog', '?']

Agora efetivamente indexa os artigos segmentados:

In [17]:
def salvar_indice_em_arquivo_pickle(indice_invertido):
  nome_arquivo_base = f"{DIR_AULA8}{ARQUIVO_INDICE_INVERTIDO}"

  with open(f'{nome_arquivo_base}_indice.pickle', 'wb') as f:
    pickle.dump(indice_invertido.indice, f)
  with open(f'{nome_arquivo_base}_tamanho_doc.pickle', 'wb') as f:
    pickle.dump(indice_invertido.tamanho_doc, f)

def recuperar_indice_em_arquivo_pickle():
  nome_arquivo_base = f"{DIR_AULA8}{ARQUIVO_INDICE_INVERTIDO}"
  idx = IndiceInvertido()

  with open(f'{nome_arquivo_base}_indice.pickle', 'rb') as f:
    idx.indice = pickle.load(f)
  with open(f'{nome_arquivo_base}_tamanho_doc.pickle', 'rb') as f:
    idx.tamanho_doc = pickle.load(f)
  idx.n_docs = len(idx.tamanho_doc)

  return idx

In [18]:
%%time
iidx = IndiceInvertido()

if GERAR_INDICE_INVERTIDO:
  # Vamos usar o próprio índice como um id para o segmento. Vai facilitar a vida depois
  # art_seg['contents'] tem o título e 3 frases do artigo
  for id, art_seg in enumerate(artigos_segmentados):
    iidx.adiciona_doc(id, art_seg['contents'])

    if id % 10000 == 0:
      print(f'{id} segmentos indexados')

  salvar_indice_em_arquivo_pickle(iidx)
else:
  iidx = recuperar_indice_em_arquivo_pickle()

CPU times: user 2.5 s, sys: 522 ms, total: 3.02 s
Wall time: 5.05 s


Agora sim cria o buscador BM25 e testa:

In [19]:
buscador = BM25(iidx, 0.82, 0.68, 1)
buscador = BM25(iidx)

Faz um teste pesquisando a primeira pergunta e exibindo os 10 primeiros resultados do BM25.

In [20]:
print('Pergunta e resposta esperada: ')
print(perg_resposta_esperada[0])

resultado_bm25 = buscador.pesquisar(perg_resposta_esperada[0]['pergunta'])

print('-----------------------------------------------------------------')
print('10 primeiros conteúdos retornados pelo BM25\n')
for i in range(10):
  segmento_id = resultado_bm25[i][0]
  score = resultado_bm25[i][1]

  title = artigos_segmentados[segmento_id]['title']
  contents = artigos_segmentados[segmento_id]['contents']
  segment = artigos_segmentados[segmento_id]['segment']

  print(f'Segmento_id: {segmento_id}. Score: {score}')
  print(f"Title: {title}")
  print(f"Contents: {contents}")
  print('-----------------------------------------------------------------')

Pergunta e resposta esperada: 
{'pergunta': 'What is Zeus know for in Greek mythology?', 'resposta': 'sky and thunder god'}
-----------------------------------------------------------------
10 primeiros conteúdos retornados pelo BM25

Segmento_id: 1140. Score: 25.03773307800293
Title: Zeus
Contents: Zeus. - Rohde, Erwin, Psyche: The Cult of Souls and Belief in Immortality among the Greeks, 1925. - Smith, William, Dictionary of Greek and Roman Biography and Mythology, 1870, Ancientlibrary.com, William Smith, Dictionary: "Zeus" Ancientlibrary.com

External links.- Greek Mythology Link, Zeus stories of Zeus in myth
- Theoi Project, Zeus summary, stories, classical art
- Theoi Project, Cult Of Zeus cult and statues
- Photo: Pagans Honor Zeus at Ancient Athens Temple from National Geographic
-----------------------------------------------------------------
Segmento_id: 510. Score: 24.084765434265137
Title: Greek mythology
Contents: Greek mythology. In Ancient Roman times, a new Roman mythol

# Agregação com Llama 3

Vamos selecionar algumas perguntas/respostas pra passar no prompt:

In [21]:
def mensagem_user(numExemplo, pergunta, lista_documentos):
  doc_com_id = [ f"[Document {i+1}]: {lista_documentos[i]}" for i in range(len(lista_documentos))]
  return f"Example {numExemplo}\n\n" + "\n\n".join(doc_com_id) + f"\n\nQuestion: {pergunta}"

def mensagem_assistant(evidencia, resposta):
  return f"Evidence: {evidencia}\n\nAnswer: {resposta}"

few_shot_examples = [
    {"user": "", "assistant": ""},
    {"user": "", "assistant": ""},
]
few_shot_examples[0]["user"] = mensagem_user(1, "Did Pink Floyd have a song about the French Riviera?", [
    "\"San Tropez\" is the fourth track from the album Meddle by the band Pink Floyd. This song was one of several to be considered for the band's \"best of\" album, Echoes: The Best of Pink Floyd.",
    "The French Riviera (known in French as the Côte d'Azur [kot daˈzyʁ]; Occitan: Còsta d'Azur [ˈkɔstɔ daˈzyɾ]; literal translation \"Azure Coast\") is the Mediterranean coastline of the southeast corner of France. There is no official boundary, but it is usually considered to extend from Cassis, Toulon or Saint-Tropez on the west to Menton at the France–Italy border in the east, where the Italian Riviera joins. The coast is entirely within the Provence-Alpes-Côte d'Azur (Région Sud) region of France. The Principality of Monaco is a semi-enclave within the region, surrounded on three sides by France and fronting the Mediterranean.",
    "Moon also promised transparency in his presidency, moving the presidential residence from the palatial and isolated Blue House to an existing government complex in downtown Seoul.",
    "Saint-Tropez (US: /ˌsæn troʊˈpeɪ/ SAN-troh-PAY, French: [sɛ̃ tʁɔpe]; Occitan: Sant-Tropetz , pronounced [san(t) tʀuˈpes]) is a town on the French Riviera, 68 kilometres (42 miles) west of Nice and 100 kilometres (62 miles) east of Marseille in the Var department of the Provence-Alpes-Côte d'Azur region of Occitania, Southern France."
])
few_shot_examples[0]["assistant"] = mensagem_assistant("\"San Tropez\" is a song by Pink Floyd about the French Riviera [Document 1]. Saint-Tropez is a town on the French Riviera [Document 4]", "yes")
few_shot_examples[1]["user"] = mensagem_user(2, "On what date did the Loma Prieta earthquake occur?", [
    "The 1989 Loma Prieta earthquake occurred on California’s Central Coast on October 17 at local time (1989-10-18 00:04 UTC). The shock was centered in The Forest of Nisene Marks State Park approximately 10 mi northeast of Santa Cruz on a section of the San Andreas Fault System and was named for the nearby Loma Prieta Peak in the Santa Cruz Mountains. With an magnitude of 6.9 and a maximum Modified Mercalli intensity of IX (Violent), the shock was responsible for 63 deaths and 3,757 injuries.",
    "'Foreshocks. The 5.3 June 1988 and the 5.4 August 1989 events also occurred on previously unknown oblique reverse faults and were within 3 mi of the M6.9 Loma Prieta mainshock epicenter, near the intersection of the San Andreas and Sargent faults. Total displacement for these shocks was relatively small (approximately 4 in of strike-slip and substantially less reverse-slip) and although they occurred on separate faults and well before the mainshock, a group of seismologists considered these to be foreshocks due to their location in space and time relative to the main event.'",
    "The June 27, 1988, shock occurred with a maximum intensity of VI (Strong). Its effects included broken windows in Los Gatos, and other light damage in Holy City, where increased flow was observed at a water well. Farther away from the Santa Cruz Mountains, pieces of concrete fell from a parking structure at the Sunnyvale Town Center, a two-level shopping mall in Santa Clara County.",
    "The 6.9 Loma Prieta earthquake occurred on October 17, 1989. The rupture was related to the San Andreas fault system and affected the entire San Francisco Bay Area with a maximum Mercalli intensity of IX (Violent). Many structures in Oakland were badly damaged including the double-decker portion of Interstate 880 that collapsed.",
])
few_shot_examples[1]["assistant"] = mensagem_assistant("According to [Document 1] and [Document 4], the Loma Prieta earthquake occur on October 17, 1989", "1989-10-17")

initial_messages=[
    {"role": "system", "content": "You are a helpful reader who reads a list of documents and uses them as evidence to answer a question given by the user. For each example, you will write an explanation for the answer based on the documents and the question. Then, after explaining your reasoning, you will give your final answer, which should be short (less than 10 words)."},
    {"role": "user", "content": few_shot_examples[0]["user"]},
    {"role": "assistant", "content": few_shot_examples[0]["assistant"]},
    {"role": "user", "content": few_shot_examples[1]["user"]},
    {"role": "assistant", "content": few_shot_examples[1]["assistant"]}
]

Agora chamar o Llama e pegar a resposta:

In [22]:
def get_passagens(pergunta, n=5):
  passagens = []
  resultado_bm25 = buscador.pesquisar(pergunta)

  for i in range(min(len(resultado_bm25), n)):
    segmento_id = resultado_bm25[i][0]
    segment = artigos_segmentados[segmento_id]['segment']
    passagens.append(segment)

  return passagens


In [23]:
# Teste de como chamar o mensagem_user usando o retorno do BM25:

teste_pergunta = test_set[0]['questions'][0]['question']
teste_passagens = get_passagens(teste_pergunta, 5)
print(mensagem_user(3, teste_pergunta, teste_passagens))

Example 3

[Document 1]: - Rohde, Erwin, Psyche: The Cult of Souls and Belief in Immortality among the Greeks, 1925. - Smith, William, Dictionary of Greek and Roman Biography and Mythology, 1870, Ancientlibrary.com, William Smith, Dictionary: "Zeus" Ancientlibrary.com

External links.- Greek Mythology Link, Zeus stories of Zeus in myth
- Theoi Project, Zeus summary, stories, classical art
- Theoi Project, Cult Of Zeus cult and statues
- Photo: Pagans Honor Zeus at Ancient Athens Temple from National Geographic

[Document 2]: In Ancient Roman times, a new Roman mythology was born through syncretization of numerous Greek and other foreign gods. This occurred because the Romans had little mythology of their own, and inheritance of the Greek mythological tradition caused the major Roman gods to adopt characteristics of their Greek equivalents. The gods Zeus and Jupiter are an example of this mythological overlap.

[Document 3]: Greek pantheon. According to Classical-era mythology, after th

In [24]:
from groq import Groq

client = Groq(api_key=GROQ_API)

def chat_completion(pergunta, n=5):
    # Documentação: https://console.groq.com/docs/text-chat
    messages = []
    # Adiciona as mensagens de sistema e few-shot
    messages.extend(initial_messages)
    # Sempre adiciona o prompt do usuário
    passagens = get_passagens(pergunta, n)
    user_message = mensagem_user(3, pergunta, passagens)
    messages.append({
        # Set a user message for the assistant to respond to.
        "role": "user",
        "content": user_message
    })
    chat_completion = client.chat.completions.create(
        messages=messages,

        # The language model which will generate the completion.
        model="llama3-70b-8192",

        # Controls randomness: lowering results in less random completions.
        # As the temperature approaches zero, the model will become deterministic
        # and repetitive.
        temperature=0,

        # The maximum number of tokens to generate. Requests can use up to
        # 32,768 tokens shared between prompt and completion.
        max_tokens=1024,

        # Controls diversity via nucleus sampling: 0.5 means half of all
        # likelihood-weighted options are considered.
        top_p=1,

        # A stop sequence is a predefined or user-specified text string that
        # signals an AI to stop generating content, ensuring its responses
        # remain focused and concise. Examples include punctuation marks and
        # markers like "[end]".
        stop=None,

        # If set, partial message deltas will be sent.
        stream=False,
    )
    # Retorna o texto da primeira mensagem e, caso seja necessário usar depois,
    # todo o objeto
    return chat_completion.choices[0].message.content, chat_completion

In [25]:
perg_resp_esperada_obtida = []

In [26]:
from tqdm import tqdm

def salvar_perg_resp_esperada_obtida(perg_resp_esperada_obtida):
  nome_arquivo = f"{DIR_AULA8}{ARQUIVO_AGREGADO}"
  with open(nome_arquivo, 'wb') as f:
    pickle.dump(perg_resp_esperada_obtida, f)

def recuperar_perg_resp_esperada_obtida():
  nome_arquivo_base = f"{DIR_AULA8}{ARQUIVO_AGREGADO}"
  perg_respostas = []
  with open(nome_arquivo_base, 'rb') as f:
    perg_respostas = pickle.load(f)
  return perg_respostas

In [27]:
import time

# Do jeito que foi implementado, se der problema (rate limit), basta executar a
# célula novamente. Ele vai continuar de onde parou
if AGREGAR_DOCUMENTOS:
  for perg_resp in tqdm(perg_resposta_esperada[len(perg_resp_esperada_obtida):TOTAL_PERGUNTAS]):
    pergunta = perg_resp['pergunta']
    resposta_esperada = perg_resp['resposta']
    resposta_obtida = chat_completion(pergunta)

    perg_resp_esperada_obtida.append({
        "pergunta": pergunta,
        "resposta_esperada": resposta_esperada,
        "resposta_obtida": resposta_obtida
    })
    time.sleep(10)
  salvar_perg_resp_esperada_obtida(perg_resp_esperada_obtida)
else:
  perg_resp_esperada_obtida = recuperar_perg_resp_esperada_obtida()

# Cálculo do F1

In [28]:
# Funções normize_answer e token_f1_score foram retiradas do Visconde (arquivo qasper_evaluator.py)
import re
import string

def normalize_answer(s):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  Lower text and remove punctuation, articles and extra whitespace.
  """

  def remove_articles(text):
      return re.sub(r"\b(a|an|the)\b", " ", text)

  def white_space_fix(text):
      return " ".join(text.split())

  def remove_punc(text):
      exclude = set(string.punctuation)
      return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
      return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def token_f1_score(prediction, ground_truth):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  """
  prediction_tokens = normalize_answer(prediction).split()
  ground_truth_tokens = normalize_answer(ground_truth).split()
  common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
  num_same = sum(common.values())
  if num_same == 0:
      return 0
  precision = 1.0 * num_same / len(prediction_tokens)
  recall = 1.0 * num_same / len(ground_truth_tokens)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def extrai_resposta(resp):
  idx_resposta_llm = resp.find('Answer: ') + 8
  return resp[idx_resposta_llm:]

# Trata respostas vazias do GPT-3.5
def uniformiza_resposta_vazia(resp):
  valores_possiveis = ["n/a", "it is not possible to determine", "no information in the documents", "no information provided", "none of the documents", "documents do not provide", "unknown", "cannot be answered", "not mentioned", "not specified", "n/a", "not available", "insufficient data", "not enough information", "cannot be determined", "there is no mention", "no information available"]

  if resp == '':
    return 'none'

  resp = resp.lower()

  for valor_none in valores_possiveis:
    if valor_none in resp:
      return 'none'

  return resp

In [29]:
import pandas as pd

col_queries = []
col_resp_esperadas = []
col_respostas_llm = []
col_f1_score = []
col_em_score = []

for perg_respostas in perg_resp_esperada_obtida:
  query = perg_respostas['pergunta']
  resposta_esperada = normalize_answer(perg_respostas['resposta_esperada'])
  resposta_llm = normalize_answer(uniformiza_resposta_vazia(extrai_resposta(perg_respostas['resposta_obtida'][0])))
  f1_score = token_f1_score(resposta_llm, resposta_esperada)
  em_score = 1 if resposta_llm == resposta_esperada else 0

  #print(f'{query}\n{resposta_esperada}\n{resposta_llm}\n---------------------------------------------')

  col_queries.append(query)
  col_resp_esperadas.append(resposta_esperada)
  col_respostas_llm.append(resposta_llm)
  col_f1_score.append(f1_score)
  col_em_score.append(em_score)

df = pd.DataFrame({'Query': col_queries, 'Resp_esperada': col_resp_esperadas, 'Resposta_LLM': col_respostas_llm, 'F1_score': col_f1_score, 'EM_score': col_em_score})
df

,Query,Resp_esperada,Resposta_LLM,F1_score,EM_score
0,What is Zeus know for in Greek mythology?,sky and thunder god,being king of gods,0.000000,0
1,How long had the First World War been over whe...,5,5 years,0.666667,0
2,How long had Angela Scoular been acting profes...,2,at least 2 years,0.400000,0
3,What is the capacity of the stadium where Brun...,26688,none,0.000000,0
4,In which country was Wilhelm Müller born?,germany,germany,1.000000,1
...,...,...,...,...,...
145,Which of the destinations had the largest popu...,none,new york city,0.000000,0
146,Was Rao alive when Manmohan Singh was prime mi...,yes,no,0.000000,0
147,Who was in charge of the London County Council...,municipal reformers,none,0.000000,0
148,How many years passed between the sack of Cons...,4,4 years,0.666667,0


Exibe o F1 e o EM (exact match) médios:

In [36]:
print(f'Total de exemplos: {len(df)}')
print(f"F1 score. Média: {df['F1_score'].mean():.2f}. Std: {df['F1_score'].std():.2f}")
print(f"EM score. Média: {df['EM_score'].mean():.2f}. Std: {df['EM_score'].std():.2f}")

Total de exemplos: 150
F1 score. Média: 0.46. Std: 0.46
EM score. Média: 0.38. Std: 0.49


Agora considera apenas os dados diferentes de None:

In [37]:
df_com_resposta = df[df.Resp_esperada != "none"]
print(f'Total de exemplos: {len(df_com_resposta)}')
print(f"F1 score. Média: {df_com_resposta['F1_score'].mean():.2f}. Std: {df_com_resposta['F1_score'].std():.2f}")
print(f"EM score. Média: {df_com_resposta['EM_score'].mean():.2f}. Std: {df_com_resposta['EM_score'].std():.2f}")

Total de exemplos: 104
F1 score. Média: 0.33. Std: 0.42
EM score. Média: 0.21. Std: 0.41
